In [1]:
import pickle
import numpy as np
import tensorflow as tf

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

In [3]:
#load the CIFAR-10
def load_CIFAR10(pos, n_chunks=1):
    Xtr = []
    Ytr = []
    for i in range(n_chunks):
        train = unpickle(pos + '/data_batch_{0}'.format(i + 1))
        Xtr.extend(train[b'data'])
        Ytr.extend(train[b'labels'])
        test = unpickle(pos + '/test_batch')
        Xte = test[b'data']
        Yte = test[b'labels']
    return np.array(Xtr), np.array(Ytr), np.array(Xte), np.array(Yte)

In [4]:
# expresses the label data in one-hot encoding.
def onehot_encoding (Ytr, Yte):
    Ytr_onehot = np.zeros((Ytr.size, 10))
    Yte_onehot = np.zeros((Yte.size, 10))
    for i in range(Ytr.size):
        Ytr_onehot[i][Ytr[i]] = 1
    for i in range(Yte.size):
        Yte_onehot[i][Yte[i]] = 1
    return Ytr_onehot, Yte_onehot

In [5]:
# load the train and test data
Xtr, Ytr, Xte, Yte = load_CIFAR10('cifar-10-batches-py', 5)
                                 
# image data, each data size is 32*32*3
Xtr = Xtr.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype("float")
Xte= Xte.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")

# label data of train and test data, label data is represented by one-hot encoding
Ytr_onehot, Yte_onehot = onehot_encoding(Ytr, Yte)

In [6]:
with tf.device('/gpu:0'):
    X = tf.placeholder(tf.float32, [None, 32, 32, 3])
    Y = tf.placeholder(tf.float32, [None, 10])
    dropout_prob = tf.placeholder(tf.float32)

### Implement the layers of CNNs

In [7]:
with tf.name_scope('Conv-1'):
    W1 = tf.Variable(tf.random_normal([3, 3, 3, 32], stddev=0.01))
    # padding='SAME' keeps output size equal to input
    L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
    # Batch normalization
    batch_mean, batch_var = tf.nn.moments(L1, [0])
    scale = tf.Variable(tf.ones([32, 32, 32]))
    beta = tf.Variable(tf.zeros([32, 32, 32]))
    L1 = tf.nn.batch_normalization(L1, batch_mean, batch_var, beta, scale, 1e-3)
    L1 = tf.nn.relu(L1)
    # Pooling layer
    L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [8]:
with tf.name_scope('Conv-2'):
    W2 = tf.Variable(tf.random_normal([3, 3, 32, 128], stddev=0.01))
    L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
    batch_mean, batch_var = tf.nn.moments(L2, [0])
    scale = tf.Variable(tf.ones([16, 16, 128]))
    beta = tf.Variable(tf.zeros([16, 16, 128]))
    L2 = tf.nn.batch_normalization(L2, batch_mean, batch_var, beta, scale, 1e-3)

    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [9]:
with tf.name_scope('Conv-3'):
    W3 = tf.Variable(tf.random_normal([5, 5, 128, 256], stddev=0.01))
    L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
    batch_mean, batch_var = tf.nn.moments(L3, [0])
    scale = tf.Variable(tf.ones([8, 8, 256]))
    beta = tf.Variable(tf.zeros([8, 8, 256]))
    L3 = tf.nn.batch_normalization(L3, batch_mean, batch_var, beta, scale, 1e-3)

    L3 = tf.nn.relu(L3)
    L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [10]:
with tf.name_scope('Full-1'):
    W5 = tf.Variable(tf.random_normal([4 * 4 * 256, 512], stddev=0.01))
    L5 = tf.reshape(L3, [-1, 4 * 4 * 256])
    L5 = tf.matmul(L5, W5)
    batch_mean, batch_var = tf.nn.moments(L5, [0])
    scale = tf.Variable(tf.ones([512]))
    beta = tf.Variable(tf.zeros([512]))
    L5 = tf.nn.batch_normalization(L5, batch_mean, batch_var, beta, scale, 1e-3)
    L5 = tf.nn.relu(L5)
    # Dropout
    L5 = tf.nn.dropout(L5, dropout_prob)

In [11]:
with tf.name_scope('Full-2'):
    W6 = tf.Variable(tf.random_normal([512, 10], stddev=0.01))
    model = tf.matmul(L5, W6)

In [12]:
# cost function, you can change the implementation
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001, 0.9).minimize(cost)

In [13]:
# initialize the variables
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

### Implement train and test processes

In [14]:
mini_batch = 100
total_batch = int(len(Xtr) / mini_batch)

In [15]:
def next_batch(start, end):
    batch_xs = Xtr[start:end]
    batch_ys = Ytr_onehot[start:end]
    return batch_xs, batch_ys

In [16]:
for epoch in range(100):
    total_cost = 0
    
    batch_start = 0
    batch_end = mini_batch
    for i in range(total_batch):
        batch_xs, batch_ys = next_batch(batch_start, batch_end)
#         batch_xs = batch_xs.reshape(-1, 3136, 3136, 1)    # When image number is unknown put -1
        _, curr_loss, = sess.run([optimizer, cost],
                                 feed_dict={X: batch_xs, Y: batch_ys, dropout_prob: 0.7})
        total_cost += curr_loss
        batch_start += mini_batch
        batch_end += mini_batch

    correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
    print('Epoch: ', '%02d' % (epoch + 1),
          '\tAvg cost: {:,.3f}'.format(total_cost / total_batch),
          '\tTest accuracy: ', sess.run(accuracy, feed_dict={X: Xte,
                                                             Y: Yte_onehot,
                                                             dropout_prob: 1.0}))    # No dropou

Epoch:  01 	Avg cost: 1.124 	Test accuracy:  0.7047
Epoch:  02 	Avg cost: 0.758 	Test accuracy:  0.7445
Epoch:  03 	Avg cost: 0.576 	Test accuracy:  0.7553
Epoch:  04 	Avg cost: 0.434 	Test accuracy:  0.7586
Epoch:  05 	Avg cost: 0.308 	Test accuracy:  0.7614
Epoch:  06 	Avg cost: 0.221 	Test accuracy:  0.7608
Epoch:  07 	Avg cost: 0.157 	Test accuracy:  0.7599
Epoch:  08 	Avg cost: 0.119 	Test accuracy:  0.7605
Epoch:  09 	Avg cost: 0.103 	Test accuracy:  0.759
Epoch:  10 	Avg cost: 0.084 	Test accuracy:  0.7635
Epoch:  11 	Avg cost: 0.068 	Test accuracy:  0.7747
Epoch:  12 	Avg cost: 0.058 	Test accuracy:  0.7704
Epoch:  13 	Avg cost: 0.054 	Test accuracy:  0.77
Epoch:  14 	Avg cost: 0.054 	Test accuracy:  0.7677
Epoch:  15 	Avg cost: 0.056 	Test accuracy:  0.7636
Epoch:  16 	Avg cost: 0.046 	Test accuracy:  0.7675
Epoch:  17 	Avg cost: 0.041 	Test accuracy:  0.7703
Epoch:  18 	Avg cost: 0.038 	Test accuracy:  0.7652
Epoch:  19 	Avg cost: 0.045 	Test accuracy:  0.769
Epoch:  20 	Avg 